In [23]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

In [24]:
awg.awg._ch[1].sync_marker_high=1.5

In [33]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})

seq.awg =awg
seq.start_new_segment('Starting segment')

seq.asc(length_mus = 10, green=True, orange = True)

# seq.asc(pdawg1=dict(type='sine', frequencies=[1750.], amplitudes=[0.9]),
#         length_mus=10, green=True, orange = True)

seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000wait    1       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [34]:
seq.write_seq()

MCAS 'Laser' has been generated.


In [35]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0180 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0230 seconds


In [29]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0150 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0110 seconds


## pODMR

In [1]:
2.87 + 2.87- 1.9

3.8400000000000003

In [30]:
AWG_M8190A_qudi.awg._ch[1].output_amplitude

0.5

In [14]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.7

def awg_pODMR(freqs):
    seq = mcas.MultiChSeq('hahn_echo', ch_dict={'awg': [1, 2]})
#     phases = [0.]
    electron_amp = 0.03 # AU 300mV peak to peak
    electron_pi = 1.550/2 # us
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
    for freq in freqs:
        seq.start_new_segment('hahn')
        seq.asc(pdawg1=dict(type='sine', frequencies=[freq], amplitudes=[electron_amp]),
                length_mus=electron_pi)
        seq.asc(length_mus=0.1)
        seq = awg_AddReadout(seq)
        seq.asc(length_mus=40)

        
    return seq

# def awg_AddReadout(seq):
#     length_laserreadout_ns = 3000
#     length_lasersafety_ns = 1000
#     length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    
#     while length_laserreadout_smpl % 64 != 0:
#         length_laserreadout_smpl += 1
        
#     seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)
    
#     seq.asc(length_mus= length_lasersafety_ns / 1e3)
#     return seq


def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 500
    AOMdelay_ns = 400
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    AOMdelay_sample = AOMdelay_ns  / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(length_mus = 10, green=False, orange = True) #green on, AOM off

    seq.asc(timetagger=False,green=True,orange=False, length_smpl=AOMdelay_sample)
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)    
    seq.asc(timetagger=True,green=False,orange=True, length_smpl=AOMdelay_sample)

    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq




def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq

freqs = np.arange(1825., 1855, .05)


seq_pODMR = awg_pODMR(freqs)
seq_pODMR.awg = awg




# pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
#                                            controlled_variable= taus,
#                                            number_of_lasers=2*len(taus), 
#                                            laser_ignore_list=[], 
#                                            alternating=True, 
#                                            units=('s', 'arb. u.'))




In [15]:
len(freqs)


600

In [16]:

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= freqs,
                                           number_of_lasers=len(freqs), 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('MHz', 'arb. u.'))

In [17]:
# seqEcho.sequences['awg'][1].print_info()

In [18]:
seq_pODMR.write_seq()

MCAS 'hahn_echo' has been generated.


In [19]:
seq_pODMR.run_sequence()

writing sequence hahn_echo on AWG_M8190A_qudi ch 1 took 16.1493 seconds
writing sequence hahn_echo on AWG_M8190A_qudi ch 2 took 15.9432 seconds


In [22]:
seq_pODMR.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0130 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0120 seconds


33.333333333333336